In [1]:
import numpy as np 
import pandas as pd 
import nltk
from nltk.corpus import stopwords
import string
import os
import re
import matplotlib.pyplot as plt

C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [3]:
os.chdir("D:\OneDrive - Manipal Global Education Services Pvt Ltd\Official\MGAIT\Datasets")

In [5]:
df = pd.read_csv('emails.txt') #read the CSV file

In [6]:
df.head(5)

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


In [7]:
df.shape

(5728, 2)

In [53]:
df.spam.value_counts(normalize=True)

0    0.759789
1    0.240211
Name: spam, dtype: float64

In [9]:
df.columns

Index(['text', 'spam'], dtype='object')

In [10]:
df.drop_duplicates(inplace = True)

In [11]:
df.shape

(5695, 2)

In [12]:
df.isnull().sum()

text    0
spam    0
dtype: int64

In [13]:
from string import punctuation
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import SnowballStemmer

In [14]:
def clean_txt(sent):
    #Stripping white spaces before and after the text
    sent = sent.strip()
    #Replacing multiple spaces with a single space
    result = re.sub("\s+", " ", sent)
    #Replacing Non-Alpha-numeric and non space charecters with nothing
    result1 = re.sub("[^\w\s]","",result)
    tokens = word_tokenize(sent.lower())
    stop_updated = stopwords.words("english")  +  ["would", "could","told","subject"]
    text = [term for term in tokens if term not in stop_updated and len(term) > 2] 
    res = " ".join(text)
    return res

In [15]:
df['text'] = df.text.apply(clean_txt)

In [16]:
df.head()

,text,spam
0,naturally irresistible corporate identity real...,1
1,stock trading gunslinger fanny merrill muzo co...,1
2,unbelievable new homes made easy wanting show ...,1
3,color printing special request additional info...,1
4,money get software cds software compatibility ...,1


In [17]:
#Seperate text column and the labels into X and y
X_text = df.text.values
y = df.spam.values

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=3500)
X = tfidf_vectorizer.fit_transform(X_text)

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state=42)

In [20]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [21]:
y_pred = classifier.predict(X_test)

In [22]:
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

In [23]:
accuracy_score(y_test,y_pred)

0.9824458747805734

In [24]:
confusion_matrix(y_test,y_pred)

array([[1263,    6],
       [  24,  416]], dtype=int64)

In [25]:
f1_score(y_test,y_pred)

0.9651972157772621

In [39]:
New_Email = pd.DataFrame([["undelivered email error"],["visiting bangalore, on leave"]], columns = ["text"])
New_Email

,text
0,undelivered email error
1,"visiting bangalore, on leave"


In [45]:
New_Email['clean_text'] = New_Email.text.apply(clean_txt)

In [50]:
New_Email.clean_text.values

array(['undelivered email error', 'visiting bangalore leave'],
      dtype=object)

In [51]:
newemail_tfidf = tfidf_vectorizer.transform(New_Email.clean_text.values)

In [52]:
classifier.predict(newemail_tfidf)

array([1, 0], dtype=int64)